In [ ]:
import pandas as pd
import numpy as np
import re

df = pd.read_csv('IMDB_Dataset.csv')

# basic preprocess
def preprocess(text):
    text = text.lower()
    text = re.sub(r'[^a-z0-9\s]', '', text)
    return text

df['clean'] = df['review'].astype(str).apply(preprocess)
df['positive'] = (df['sentiment'] == 'positive').astype(int)

P_positive = df['positive'].mean()
total = len(df)

keywords = ['excellent','great','amazing','boring','awful','terrible']  # 3 positive and 3 negative keywords

rows = []
for w in keywords:
    contains = df['clean'].str.contains(r'\b' + re.escape(w) + r'\b', regex=True)
    count_contains = contains.sum()
    # likelihood: P(w | Positive)
    if df['positive'].sum() > 0:
        P_w_given_pos = contains & (df['positive']==1)
        P_w_given_pos = P_w_given_pos.sum() / df['positive'].sum()
    else:
        P_w_given_pos = 0
    P_w = count_contains / total
    if P_w > 0:
        posterior = (P_w_given_pos * P_positive) / P_w
    else:
        posterior = np.nan
    rows.append((w, P_positive, P_w_given_pos, P_w, posterior))

table = pd.DataFrame(rows, columns=['keyword','P(Positive)','P(keyword|Positive)','P(keyword)','P(Positive|keyword)'])
table

ValueError: mount failed